<a href="https://colab.research.google.com/github/LBncl/AmazonMLInterviewQuestion/blob/main/NAFLD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NAFLD PROJECT

## Imports

In [1]:
# Imports
import missingno as msno
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Data Cleaning

In [53]:
# Read in data
df_raw = pd.read_excel('master_with_nordic_and_multiBM.xlsx')

# Check if dataFrame is empty
if df_raw.empty:
    print('DataFrame is empty!')

In [90]:
# Subset data frame
main_df = df_raw.filter(items=['CPH_EV_AGE_CALC', 'TBL.PATIENT.INFO..PI_BL_GENDER', 'CPH_EV_CI_BMI_CALC', 
                         'TBL.ALL.EVENTS..AE_SF_ALCO_XS', 'insulin_resistance', 'hypertensive', 'waist_to_hip_ratio',
                         'idf_metabolic_syndrome', 'eGFR', 'dyslipidaemia', 'fibroscan_stiffness_reliable',
                         'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
                         'TBL.ALL.EVENTS..AE_BR_AST_iuL', 'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
                         'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
                         'TBL.ALL.EVENTS..AE_BR_PLT_109L', 'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
                         'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
                         'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC', 'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
                         'TBL.ALL.EVENTS..AE_BR_IGA',
                         'TBL.ALL.EVENTS..AE_CD_OSA', 'LIT_NB_CK18_M30', 'LIT_NB_CK18_M65', 'LIT_NB_PRO_C3', 'LIT_NB_PRO_C6',
                          'LIT_NB_ELF', 'FIB4', 'NFS', 'APRI', 'ADAPT', 'FIBC3', 'ABC3D', 'BARD', 'AST_ALT_Ratio', 'response_7'])

In [ ]:
main_df.describe()

In [ ]:
main_df.dtypes

In [ ]:
msno.bar(main_df)

In [91]:
# Convert required variables to boolean and normlaise the remaining variables
features_raw = main_df.iloc[:, 0:35]

# Loop to check for boolean columns
for column in features_raw:
  if features_raw[column].max() == 1 and features_raw[column].min() == 0:
    features_raw[column] = features_raw[column].astype(bool)

In [ ]:
features_raw.dtypes

In [92]:
# Filter to only select basline event types
features_raw['CPH_EV_EVENT_TYPE'] = df_raw['CPH_EV_EVENT_TYPE']
is_Baseline = features_raw['CPH_EV_EVENT_TYPE'] == 'Baseline'
features_baseline = features_raw[is_Baseline]
features_baseline = features_baseline.iloc[: , :-1]
features_baseline = features_baseline.reset_index()
features_baseline = features_baseline.iloc[: , 1:]

In [ ]:
features_baseline

In [ ]:
# remove outliers
import numpy as np
from scipy import stats
cols_normalise = features_baseline.select_dtypes(include=[np.float64])

z_scores = stats.zscore(features_baseline[cols_normalise.columns],  nan_policy = 'omit')
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
features_filtered = features_baseline[cols_normalise.columns]

In [30]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

cols_normalise = features_baseline.select_dtypes(include=[np.float64])

# Normalise Data
scale = StandardScaler()
features_baseline[cols_normalise.columns] = scale.fit_transform(cols_normalise)

In [ ]:
features_baseline[cols_normalise.columns]

In [93]:
from sklearn import preprocessing
names = list(features_baseline.columns)

x = features_baseline.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
features_baseline_scaled = min_max_scaler.fit_transform(x)
features_baseline_scaled = pd.DataFrame(features_baseline_scaled, columns=names)
features_baseline_scaled

,CPH_EV_AGE_CALC,TBL.PATIENT.INFO..PI_BL_GENDER,CPH_EV_CI_BMI_CALC,TBL.ALL.EVENTS..AE_SF_ALCO_XS,insulin_resistance,hypertensive,waist_to_hip_ratio,idf_metabolic_syndrome,eGFR,dyslipidaemia,...,LIT_NB_PRO_C6,LIT_NB_ELF,FIB4,NFS,APRI,ADAPT,FIBC3,ABC3D,BARD,AST_ALT_Ratio
0,0.483871,1.0,0.688333,0.0,1.0,0.0,NaN,0.0,0.372273,0.0,...,NaN,NaN,0.046102,0.712414,0.020094,NaN,NaN,0.500000,1.00,0.155070
1,0.483871,0.0,0.165333,0.0,1.0,1.0,0.228253,0.0,NaN,1.0,...,NaN,NaN,0.299008,0.445626,0.193066,NaN,NaN,0.500000,0.25,0.094718
2,0.451613,1.0,0.490333,1.0,0.0,0.0,NaN,0.0,0.206149,0.0,...,NaN,NaN,0.049002,0.393620,0.012570,NaN,NaN,0.166667,0.25,0.115076
3,0.451613,0.0,0.681333,1.0,0.0,0.0,NaN,1.0,0.270558,1.0,...,NaN,NaN,0.063383,0.491051,0.018272,NaN,NaN,0.166667,0.25,0.112998
4,0.467742,1.0,0.458333,1.0,0.0,1.0,NaN,1.0,0.266043,1.0,...,NaN,NaN,0.035208,0.383990,0.012686,NaN,NaN,0.166667,0.75,0.206343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9283,0.661290,1.0,0.386333,0.0,1.0,1.0,NaN,0.0,NaN,1.0,...,NaN,NaN,NaN,0.631983,NaN,NaN,NaN,0.666667,NaN,NaN
9284,0.354839,1.0,0.393333,0.0,0.0,0.0,0.229013,0.0,NaN,1.0,...,NaN,NaN,NaN,0.303982,NaN,NaN,NaN,NaN,NaN,NaN
9285,0.677419,1.0,0.356667,0.0,0.0,0.0,NaN,1.0,0.600826,1.0,...,NaN,NaN,0.103404,0.419137,0.046431,NaN,NaN,0.333333,0.75,0.184740
9286,0.677419,1.0,0.443667,0.0,1.0,1.0,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,0.667922,NaN,NaN,NaN,NaN,NaN,NaN


## Data Imputation Strategies

In [34]:
# Imports for imputation functions
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression

In [35]:
# Single imputation with mean to replace not a number (NaNs)
def nan2mean(fdf):
    cols = list(fdf.columns)
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    fdf=imp.fit_transform(fdf)
    fdf = pd.DataFrame(fdf, columns=cols)
    return fdf

In [36]:
# Single imputation with median to replace not a number (NaNs)
def nan2median(fdf):
    cols = list(fdf.columns)
    imp = SimpleImputer(missing_values=np.nan, strategy='median')
    fdf=imp.fit_transform(fdf)
    fdf = pd.DataFrame(fdf, columns=cols)
    return fdf

In [37]:
# Single imputation with most_frequent to replace not a number (NaNs)
def nan2most_frequent(fdf):
    cols = list(fdf.columns)
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    fdf=imp.fit_transform(fdf)
    fdf = pd.DataFrame(fdf, columns=cols)
    return fdf

In [38]:
# Multiple Imputation by chained equation linear regression
def nan2mice(fdf):
    cols = list(fdf.columns)
    lr = LinearRegression()
    imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=50, imputation_order='roman',random_state=0)
    fdf=imp.fit_transform(fdf)
    fdf = pd.DataFrame(fdf, columns=cols)
    return fdf

In [39]:
# Imputation by K neasrest neighbours
def nan2knn(fdf):
    cols = list(fdf.columns)
    imp = KNNImputer(n_neighbors=2, weights="distance")
    fdf=imp.fit_transform(fdf)
    fdf = pd.DataFrame(fdf, columns=cols)
    return fdf

## Model evaluation

In [ ]:
# Create new imputed datasets
featuresKNN = nan2knn(features_baseline_scaled)
featuresMean = nan2mean(features_baseline_scaled)
featuresMedian = nan2median(features_baseline_scaled)
featuresMICE = nan2mice(features_baseline_scaled)
featuresMostFrq = nan2most_frequent(features_baseline_scaled)
featuresMostFrq = featuresMostFrq.apply(pd.to_numeric)

# Create list of all imputation strategies
imputed_features = [featuresKNN,featuresMean,featuresMedian,featuresMICE,featuresMostFrq]
imputed_features_names = ['featuresKNN','featuresMean','featuresMedian','featuresMICE','featuresMostFrq']

In [ ]:
column_name = "CPH_EV_AGE_CALC"

d = {'raw': features_baseline[column_name], 
     'KNN Imputation': featuresKNN[column_name],
     'Mean Imputation': featuresMean[column_name],
     'Median Imputation': featuresMedian[column_name],
     'MICE Imputation': featuresMICE[column_name],
     'MostFrq Imputation': featuresMostFrq[column_name]}
fdata = pd.DataFrame(data=d)

# calling density() to make multiple density plot 
fig, axes = plt.subplots(1, 6)
fig.suptitle('Raw vs Imputed Desity Plots')
fdata['raw'].plot.density(ax=axes[0], figsize = (15, 7))
fdata['Mean Imputation'].plot.density(ax=axes[1], figsize = (15, 7), color='red')
fdata['Median Imputation'].plot.density(ax=axes[2], figsize = (15, 7), color='red')
fdata['MICE Imputation'].plot.density(ax=axes[3], figsize = (15, 7), color='red')
fdata['MostFrq Imputation'].plot.density(ax=axes[4], figsize = (15, 7), color='red')
fdata['KNN Imputation'].plot.density(ax=axes[5], figsize = (15, 7), color='red')

axes[0].set_title('Raw')
axes[1].set_title('Mean Imputed')
axes[2].set_title('Median Imputed')
axes[3].set_title('MICE Imputed')
axes[4].set_title('MostFrq Imputed')
axes[5].set_title('KNN Imputed')

axes[0].set_xlabel(column_name)
axes[1].set_xlabel(column_name)
axes[2].set_xlabel(column_name)
axes[3].set_xlabel(column_name)
axes[4].set_xlabel(column_name)
axes[5].set_xlabel(column_name)

In [95]:
# Pull response from data frame with only baseline observations
y = main_df["response_7"]
response = pd.DataFrame({'y':y, 'Baseline':df_raw['CPH_EV_EVENT_TYPE']})
is_Baseline = response['Baseline'] == 'Baseline'
response = response[is_Baseline]
response = response['y']

# Add response variable to all imputed datasets and remove observations with NaN responses
for i in range(len(imputed_features)):
  df = imputed_features[i]
  df['response'] = response
  df = df.dropna(inplace=True)

# Feature Selection

In [ ]:
pip install featurewiz

In [ ]:
# automatic feature selection by using featurewiz package
from featurewiz import featurewiz
target = 'response'

features_selected, train = featurewiz(featuresKNN, target, corr_limit=0.7, verbose=2, sep=",", header=0,test_data="", feature_engg="", category_encoders="")

In [ ]:
from sklearn.model_selection import train_test_split

# function to generate test/train split
def train_test (fdf):
  # Split data into test and trian
  X_train, X_test, y_train, y_test = train_test_split(fdf.iloc[:, 0:35], fdf.iloc[: , -1], test_size=0.30, random_state=0)
  return X_train, X_test, y_train, y_test

In [ ]:
# Filter all data frames based upon best features
featuresKNN = featuresKNN[features_selected]
featuresMean = featuresMean[features_selected]
featuresMedian = featuresMedian[features_selected]
featuresMICE = featuresMICE[features_selected]
featuresMostFrq = featuresMostFrq[features_selected]

imputed_features = [featuresKNN,featuresMean,featuresMedian,featuresMICE,featuresMostFrq]

# Add response variable to all imputed datasets
for i in range(len(imputed_features)):
  df = imputed_features[i]
  df['response'] = response

In [ ]:
featuresKNN

### Logistic Regression

In [89]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

# Logistic Regression model
# Implement the model
logreg = LogisticRegression()

# hyperparameters, penalty values chosen to work with all solvers
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=logreg, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)

for i in range(len(imputed_features)):
  df = imputed_features[i]
  grid_result = grid_search.fit(df.iloc[:, 0:35], df.iloc[: , -1])

  # summarize results
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_), imputed_features_names[i])

Best: 0.608406 using {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'} featuresKNN
Best: 0.607496 using {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'} featuresMean
Best: 0.608210 using {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'} featuresMedian
Best: 0.607690 using {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'} featuresMICE
Best: 0.607366 using {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'} featuresMostFrq


In [45]:
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

featuresMICE

,CPH_EV_AGE_CALC,TBL.PATIENT.INFO..PI_BL_GENDER,CPH_EV_CI_BMI_CALC,TBL.ALL.EVENTS..AE_SF_ALCO_XS,insulin_resistance,hypertensive,waist_to_hip_ratio,idf_metabolic_syndrome,eGFR,dyslipidaemia,...,LIT_NB_ELF,FIB4,NFS,APRI,ADAPT,FIBC3,ABC3D,BARD,AST_ALT_Ratio,response
0,48.0,1.0,40.65,0.0,1.0,0.0,0.960952,0.0,125.760131,0.0,...,9.643921,0.781035,5.05210,0.260546,6.004213,-0.959882,3.000000,4.000000,0.976744,0.0
3,46.0,0.0,40.44,1.0,0.0,0.0,1.010877,1.0,94.341646,1.0,...,7.541367,1.002893,3.82836,0.241935,4.781984,-1.414817,1.000000,1.000000,0.729730,1.0
6,64.0,1.0,43.30,1.0,1.0,1.0,0.994614,1.0,58.405469,1.0,...,8.653671,1.895490,5.89320,0.353403,3.302915,-0.316254,5.000000,4.000000,0.843750,0.0
7,51.0,0.0,44.08,1.0,1.0,1.0,1.052950,1.0,91.020595,1.0,...,7.812546,0.874661,5.48552,0.151274,3.732118,-0.874186,4.000000,2.000000,0.655172,1.0
8,39.0,1.0,44.79,1.0,0.0,0.0,0.948056,1.0,73.487334,0.0,...,7.450760,0.533494,3.97826,0.171642,5.243007,-1.533403,1.000000,3.000000,1.095238,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9281,58.0,0.0,29.60,0.0,1.0,1.0,1.029412,1.0,98.542027,1.0,...,9.206493,1.934953,4.38340,0.502546,5.257115,-0.612403,3.000000,2.370903,0.775818,1.0
9282,68.0,0.0,30.10,0.0,1.0,1.0,1.062052,1.0,82.832586,1.0,...,9.519314,1.802747,4.80040,0.293367,5.394725,-0.267306,5.000000,4.000000,0.851852,0.0
9285,60.0,1.0,30.70,0.0,0.0,0.0,0.971896,1.0,196.357198,1.0,...,10.998882,1.516690,3.43080,0.529514,6.391862,-1.157897,2.000000,3.000000,1.150943,1.0
9286,60.0,1.0,33.31,0.0,1.0,1.0,0.922746,1.0,76.963054,1.0,...,9.080536,1.752192,4.80614,0.442655,5.279172,-0.584455,3.517043,2.857170,0.894310,1.0


In [96]:
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = LogisticRegression()
# evaluate model
scores = cross_val_score(model, featuresMICE.iloc[:, 0:35], featuresMICE.iloc[: , -1], scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.657 (0.017)


### SVM

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report



### XGBoost 

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Implement the model
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

for i in range(len(imputed_features)):
  df = imputed_features[i]
  X_train, X_test, y_train, y_test = train_test(df)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  print(accuracy)



In [98]:
from sklearn.model_selection import train_test_split

model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
X_train, X_test, y_train, y_test = train_test_split(featuresMICE.iloc[:, 0:35], featuresMICE['response'], test_size=0.33, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.6622325043377675


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def model_RF(fdf, response):
  # Pull the correct resposne variable from the dataframe
  y = main_df[response]
  # Remove all rows with missing observations
  X = fdf
  X['y'] = y
  X = fdf.dropna()
  # Split data into test and trian
  X_train, X_test, y_train, y_test = train_test_split(X.iloc[:, 0:35], X.iloc[: , -1], test_size=0.20, random_state=0)
  # Implement the model
  regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
  regressor.fit(X_train, y_train)
  y_pred = regressor.predict(X_test)

  #use model to predict probability that given y value is 1
  y_pred_proba = regressor.predict_proba(X_test)[::,1]

  #calculate AUC of model
  auc = metrics.roc_auc_score(y_test, y_pred_proba)

  #print AUC score
  print(auc)

  # Calculate F-score
  report = classification_report(y_test, y_pred)
  print(report)

  return y_pred

## Model Imports

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics

# Pipeline

## Imputation and Target Variable Selection

In [ ]:
compare_denisty("CPH_EV_AGE_CALC",featuresKNN)

# New workflow

## Split

In [121]:
# Filter to only select basline event types
main_df['CPH_EV_EVENT_TYPE'] = df_raw['CPH_EV_EVENT_TYPE']
main_df_base = main_df[main_df['CPH_EV_EVENT_TYPE'] == 'Baseline']
main_df_base = main_df_base.iloc[: , :-1]

In [122]:
# Loop to check for boolean columns
for column in main_df_base:
  if main_df_base[column].max() == 1 and main_df_base[column].min() == 0:
    main_df_base[column] = main_df_base[column].astype(bool)

In [ ]:
main_df_base.dtypes

In [125]:
X_train, X_test, y_train, y_test = train_test_split(main_df_base.iloc[:, 0:35], main_df_base['response_7'], test_size=0.33, random_state=42)

## Imputing

In [ ]:
X_train_imputed = nan2mice(X_train)
X_test_imputed = nan2mice(X_test)

## Scaling 

In [135]:
from sklearn import preprocessing

to_scale_train = X_train_imputed.select_dtypes(include=[np.float64])
to_scale_test = X_train_imputed.select_dtypes(include=[np.float64])

# standardization of dependent variables
X_train_imputed_scaled = preprocessing.scale(to_scale_train)
X_test_imputed_scaled = preprocessing.scale(to_scale_test)

In [137]:
x_df_train = pd.DataFrame(X_train_imputed_scaled)
x_df_train[35] = y_train
x_df_train.dropna(inplace=True)

x_df_test = pd.DataFrame(X_test_imputed_scaled)
x_df_test[35] = y_test
x_df_test.dropna(inplace=True)

## Model

In [144]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(x_df_train.iloc[:, 0:35], x_df_train[35])
y_pred = model.predict(x_df_test.iloc[:, 0:35])
y_pred = y_pred.astype(bool)
print(classification_report(x_df_test[35].astype(bool), y_pred))

              precision    recall  f1-score   support

       False       0.60      0.93      0.73       899
        True       0.45      0.09      0.15       615

    accuracy                           0.59      1514
   macro avg       0.53      0.51      0.44      1514
weighted avg       0.54      0.59      0.49      1514

